<a href="https://colab.research.google.com/github/seohyun34/test/blob/main/summary_%2B_qa_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://zindi.africa/competitions/kenya-clinical-reasoning-challenge/data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q datasets
!pip install -q evaluate
!pip install -q rouge_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 9.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 118.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 86.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 106.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstal

In [ ]:
import pandas as pd
import torch
import numpy as np
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments, DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import Dataset
import os
from sklearn.model_selection import train_test_split
import evaluate
from datasets import Dataset, DatasetDict

# 데이터셋 준비

In [ ]:
import re
dpath = '/content/drive/MyDrive/에딥_팀프로젝트/'
dataset = pd.read_csv(dpath + 'split_aug_by_llm_train.csv')

def split_prompt(text):
    # 간호사 설명 (첫 문장)
    nurse_desc_pattern = r"^i am a nurse.*?\."
    nurse_desc_match = re.match(nurse_desc_pattern, text, re.IGNORECASE)

    if nurse_desc_match:
        nurse_desc = nurse_desc_match.group().strip()
        remaining_text = text[len(nurse_desc):].strip()
    else:
        nurse_desc = ""
        remaining_text = text.strip()

    # 질문 패턴
    question_patterns = [
        r"\bquestions?:",
        r"\b1[.)]\s+",
        r"\bq\d*[:)]",
        r"\bwhat\b",
        r"\bshould\b",
        r"\bhow\b",
        r"\bcould\b",
        r"\bwould\b",
        r"\bmay\b",
        r"\bshall\b",
        r"\?",
        r"\bcan\b"
    ]

    # 질문 위치 찾기
    question_regex = "|".join(question_patterns)
    question_match = re.search(question_regex, remaining_text)

    if question_match:
        # 질문 시작 지점
        question_start = question_match.start()
        patient_state = remaining_text[:question_start].strip()
        questions = remaining_text[question_start:].strip()
    else:
        # 질문이 없을 때
        patient_state = remaining_text.strip()
        questions = ""

    full_text = "context: " + patient_state + " " + "question: " + questions + " answer: "
    full_text = full_text.replace("\n", "").strip()
    return full_text
dataset['Prompt'] = dataset['Prompt'].apply(split_prompt)

In [ ]:
dataset.isna().sum()

,0
Master_Index,0
County,0
Health level,0
Years of Experience,400
Prompt,0
Nursing Competency,0
Clinical Panel,0
Clinician,0
GPT4.0,0
LLAMA,0


In [ ]:
import pandas as pd, numpy as np

orig_idx = np.arange(0, 400)

# reproducibility
rng = np.random.default_rng(seed=42)
val_orig_idx = rng.choice(
    orig_idx,
    size=int(0.2 * len(orig_idx)),
    replace=False
)

train_idx = dataset.index.difference(val_orig_idx) #검증용 40개
val_idx = val_orig_idx

train_df = dataset.loc[train_idx].reset_index(drop=True)
val_df = dataset.loc[val_idx].reset_index(drop=True)

print(train_df.shape, val_df.shape)

(1520, 14) (80, 14)


In [ ]:
hf_summary = DatasetDict({
    "train": Dataset.from_pandas(train_df[['Prompt', 'clinician_summary']]),
    "validation": Dataset.from_pandas(val_df[['Prompt', 'clinician_summary']]),
})

hf_management = DatasetDict({
    "train": Dataset.from_pandas(train_df[['Prompt', 'clinician_management']]),
    "validation": Dataset.from_pandas(val_df[['Prompt', 'clinician_management']]),
})

In [ ]:
from transformers import AutoTokenizer, T5ForConditionalGeneration

model_summary = AutoModelForSeq2SeqLM.from_pretrained("hossboll/clinical-t5")
tokenizer_summary = AutoTokenizer.from_pretrained("hossboll/clinical-t5")

model_management = AutoModelForSeq2SeqLM.from_pretrained("hossboll/clinical-t5")
tokenizer_management = AutoTokenizer.from_pretrained("hossboll/clinical-t5")

In [ ]:
def preprocess(examples):
    prefix = "summarize "
    # prefix + Prompt
    inputs = [f"{prefix}: " + doc for doc in examples["Prompt"]]
    targets = examples["clinician_summary"]

    # 입력 인코딩
    model_inputs = tokenizer_summary(
        inputs,
        max_length=512,
        truncation=True,
        padding="max_length",
    )

    # 타깃 인코딩
    with tokenizer_summary.as_target_tokenizer():
        labels = tokenizer_summary(
            targets,
            max_length=128,
            truncation=True,
            padding="max_length",
        )["input_ids"]

    # pad 토큰(0) → -100 로 치환
    labels = [
        [(tok if tok != tokenizer_summary.pad_token_id else -100) for tok in seq]
        for seq in labels
    ]

    model_inputs["labels"] = labels
    return model_inputs


tokenized_ds = hf_summary.map(
    preprocess,
    batched=True,
    remove_columns=hf_summary["train"].column_names,
)


Map:   0%|          | 0/1520 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/80 [00:00<?, ? examples/s]

In [ ]:
def preprocess_management(examples):
    prefix = "qa"
    # prefix + Prompt
    inputs = [f"{prefix}: " + doc for doc in examples["Prompt"]]
    targets = examples["clinician_management"]

    # 입력 인코딩
    model_inputs = tokenizer_management(
        inputs,
        max_length=512,
        truncation=True,
        padding="max_length",
    )

    # 타깃 인코딩
    with tokenizer_management.as_target_tokenizer():
        labels = tokenizer_management(
            targets,
            max_length=128,
            truncation=True,
            padding="max_length",
        )["input_ids"]

    # pad 토큰(0) → -100 로 치환
    labels = [
        [(tok if tok != tokenizer_management.pad_token_id else -100) for tok in seq]
        for seq in labels
    ]

    model_inputs["labels"] = labels
    return model_inputs


tokenized_management = hf_management.map(
    preprocess_management,
    batched=True,
    remove_columns=hf_management["train"].column_names,
)


Map:   0%|          | 0/1520 [00:00<?, ? examples/s]

Map:   0%|          | 0/80 [00:00<?, ? examples/s]

# metric

In [ ]:
rouge = evaluate.load("rouge")

def compute_metrics(eval_pred):
    preds, labels = eval_pred
    if isinstance(preds, tuple): preds = preds[0]

    # logits → token IDs
    if preds.ndim == 3:
        preds = np.argmax(preds, axis=-1)

    # Tensor → numpy
    if isinstance(preds, torch.Tensor):  preds = preds.cpu().numpy()
    if isinstance(labels, torch.Tensor): labels = labels.cpu().numpy()

    # 음수 → pad, 범위 초과 → vocab_size-1 로 컷
    vocab_max = tokenizer_summary.vocab_size - 1
    preds  = np.clip(np.where(preds  < 0, tokenizer_summary.pad_token_id, preds),  0, vocab_max)
    labels = np.clip(np.where(labels < 0, tokenizer_summary.pad_token_id, labels), 0, vocab_max)

    decoded_preds  = tokenizer_summary.batch_decode(preds.tolist(),  skip_special_tokens=True)
    decoded_labels = tokenizer_summary.batch_decode(labels.tolist(), skip_special_tokens=True)

    rouge_scores = rouge.compute(predictions=decoded_preds,
                                 references=decoded_labels,
                                 use_stemmer=True)
    return {"rougeL_f1": float(rouge_scores["rougeL"])}


In [ ]:
rouge = evaluate.load("rouge")

def compute_metrics_management(eval_pred):
    preds, labels = eval_pred
    if isinstance(preds, tuple): preds = preds[0]

    # logits → token IDs
    if preds.ndim == 3:
        preds = np.argmax(preds, axis=-1)

    # Tensor → numpy
    if isinstance(preds, torch.Tensor):  preds = preds.cpu().numpy()
    if isinstance(labels, torch.Tensor): labels = labels.cpu().numpy()

    # 음수 → pad, 범위 초과 → vocab_size-1 로 컷
    vocab_max = tokenizer_management.vocab_size - 1
    preds  = np.clip(np.where(preds  < 0, tokenizer_management.pad_token_id, preds),  0, vocab_max)
    labels = np.clip(np.where(labels < 0, tokenizer_management.pad_token_id, labels), 0, vocab_max)

    decoded_preds  = tokenizer_management.batch_decode(preds.tolist(),  skip_special_tokens=True)
    decoded_labels = tokenizer_management.batch_decode(labels.tolist(), skip_special_tokens=True)

    rouge_scores = rouge.compute(predictions=decoded_preds,
                                 references=decoded_labels,
                                 use_stemmer=True)
    return {"rougeL_f1": float(rouge_scores["rougeL"])}

# 모델링

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("hossboll/clinical-t5")
model = AutoModelForSeq2SeqLM.from_pretrained("hossboll/clinical-t5")


data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, label_pad_token_id=-100)

training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=8,
    num_train_epochs=3,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="steps",
    logging_steps = 5,
    predict_with_generate=True,
    generation_max_length=100,
    learning_rate=4e-5,
    fp16=True,
    report_to="none",
    generation_num_beams=4,
    label_smoothing_factor=0.1,
    weight_decay=0.01,
    warmup_ratio=0.05,
    optim = "adamw_torch",
    load_best_model_at_end=True,
    metric_for_best_model="rougeL_f1",
    greater_is_better=True,

)

trainer = Seq2SeqTrainer(
    model           = model,
    args            = training_args,
    train_dataset   = tokenized_ds["train"],
    eval_dataset    = tokenized_ds["validation"],
    tokenizer       = tokenizer,
    data_collator   = data_collator,
    compute_metrics = compute_metrics,
)

trainer.train()

<ipython-input-122-f87e3ec45c75>:36: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Epoch,Training Loss,Validation Loss,Rougel F1
1,4.909800,4.347051,0.283595
2,4.515000,4.060175,0.339455


KeyboardInterrupt: 

In [ ]:
model_management = T5ForConditionalGeneration.from_pretrained("t5-small")

data_collator_management = DataCollatorForSeq2Seq(tokenizer_management, model=model_management, label_pad_token_id=-100)

training_args_management = Seq2SeqTrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=8,
    num_train_epochs=10,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="steps",
    logging_steps = 5,
    predict_with_generate=True,
    generation_max_length=128,
    learning_rate=5e-5,
    fp16=True,
    report_to="none",
    generation_num_beams=4,
    label_smoothing_factor=0.1,
    weight_decay=0.01,
    warmup_ratio=0.1,
    optim = "adamw_torch",
    load_best_model_at_end=True,
    metric_for_best_model="rougeL_f1",
    greater_is_better=True,

)

trainer_management = Seq2SeqTrainer(
    model           = model_management,
    args            = training_args_management,
    train_dataset   = tokenized_management["train"],
    eval_dataset    = tokenized_management["validation"],
    tokenizer       = tokenizer_management,
    data_collator   = data_collator_management,
    compute_metrics = compute_metrics_management,
)

trainer_management.train()

<ipython-input-160-d6647e110053>:31: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer_management = Seq2SeqTrainer(


Epoch,Training Loss,Validation Loss,Rougel F1
1,6.383300,5.956678,0.084490


KeyboardInterrupt: 

# test 예측 생성

In [ ]:
save = dpath + "finetuned_t5_summary"

trainer.save_model(save) # 모델 가중치 + config 저장
tokenizer_summary.save_pretrained(save) # 토크나이저 vocab·특수토큰 저장

('/content/drive/MyDrive/에딥_팀프로젝트/finetuned_t5_summary/tokenizer_config.json',
 '/content/drive/MyDrive/에딥_팀프로젝트/finetuned_t5_summary/special_tokens_map.json',
 '/content/drive/MyDrive/에딥_팀프로젝트/finetuned_t5_summary/spiece.model',
 '/content/drive/MyDrive/에딥_팀프로젝트/finetuned_t5_summary/added_tokens.json')

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model_dir = dpath + "finetuned_t5_summary"

tokenizer = AutoTokenizer.from_pretrained(model_dir, use_fast=True)
model = T5ForConditionalGeneration.from_pretrained(model_dir).to(device)
model.eval()

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [ ]:
import torch, pandas as pd
from tqdm.auto import tqdm
prefix = 'clinical_summary'
# ── 0) 테스트 세트 로드 ──────────────────────────
test_df = pd.read_csv(dpath + 'test.csv')   # Prompt 열만 있다고 가정
print("test size =", len(test_df))

# ── 1) 파라미터 & 디바이스 설정 ───────────────

MAX_INPUT = 512
MAX_GEN = 128
BEAMS = 4
#LEN_PENALTY = 0.9
BATCH = 8                        # VRAM 여유에 맞게

# ── 2) 배치 추론 루프 ──────────────────────────
outputs = []
with torch.no_grad():
    for i in tqdm(range(0, len(test_df), BATCH)):
        batch_prompts = test_df["Prompt"].iloc[i:i+BATCH].tolist()
        batch_inputs  = [f"{prefix}: {p}" for p in batch_prompts]

        enc = tokenizer(
            batch_inputs,
            return_tensors="pt",
            padding=True,           # 가변 길이 배치
            truncation=True,
            max_length=MAX_INPUT
        ).to(device)

        gen_ids = model.generate(
            **enc,
            max_length=MAX_GEN,
            num_beams=BEAMS,
            #length_penalty=LEN_PENALTY,
        )

        batch_out = tokenizer.batch_decode(
            gen_ids, skip_special_tokens=True
        )
        outputs.extend(batch_out)

# ── 3) 결과 저장 ────────────────────────────────
test_df["Clinician_summary"] = outputs

test size = 100


  0%|          | 0/13 [00:00<?, ?it/s]

In [ ]:
test_df.loc[9, 'Prompt']

'i am a nurse with 10 years of experience in general nursing working in a national referral hospitals in uasin gishu county in kenya a 4 year old boy referred from west pokot west kapenguria county hospitalwith complains of gradual abdominal distention for 3 weeks the patient is wasted dehydrated and pale with swollen abdomen palpable mass in the abdomen no bowel sounds vitals pulse 112 blood pressure 98 50mmhg respiration rate 20b min temperature 36 7 degrees celsius spo2 96 labs cbc uecs radiology abdominal ultrasound intestinal obstruction due to worms patient prepared for emergency laparotomy and evacuation questions what are the predisposing factors to worm infestation what are the preventive measures of worm infestation what management should be instituted after surgery'

In [ ]:
test_df.loc[9, 'Clinician']

'summary a 4 year old boy referred from west pokot west kapenguria county hospital complains of gradual abdominal distention for 3 weeks the patient is wasted dehydrated and pale with swollen abdomen palpable mass in the abdomen no bowel sounds vitals pulse 112 blood pressure 98 50mmhg respiration rate 20b min temperature 36 7 celsius spo2 96 labs cbc uecs radiology abdominal ultrasound intestinal obstruction due to worms patient prepared for emergency'

In [ ]:
prefix = "clinical_management "

save = dpath + "finetuned_t5_management"

trainer_management.save_model(save) # 모델 가중치 + config 저장
tokenizer_management.save_pretrained(save) # 토크나이저 vocab·특수토큰 저장

device = "cuda" if torch.cuda.is_available() else "cpu"
model_dir = dpath + "finetuned_t5_management"

tokenizer = AutoTokenizer.from_pretrained(model_dir, use_fast=True)
model = T5ForConditionalGeneration.from_pretrained(model_dir).to(device)
model.eval()

import torch, pandas as pd
from tqdm.auto import tqdm

# ── 1) 파라미터 & 디바이스 설정 ───────────────

MAX_INPUT = 512
MAX_GEN = 128
BEAMS = 4
#LEN_PENALTY = 0.9
BATCH = 8                        # VRAM 여유에 맞게

# ── 2) 배치 추론 루프 ──────────────────────────
outputs = []
with torch.no_grad():
    for i in tqdm(range(0, len(test_df), BATCH)):
        batch_prompts = test_df["Prompt"].iloc[i:i+BATCH].tolist()
        batch_inputs  = [f"{prefix}: {p}" for p in batch_prompts]

        enc = tokenizer(
            batch_inputs,
            return_tensors="pt",
            padding=True,           # 가변 길이 배치
            truncation=True,
            max_length=MAX_INPUT
        ).to(device)

        gen_ids = model.generate(
            **enc,
            max_length=MAX_GEN,
            num_beams=BEAMS,
            #length_penalty=LEN_PENALTY,
        )

        batch_out = tokenizer.batch_decode(
            gen_ids, skip_special_tokens=True
        )
        outputs.extend(batch_out)

# ── 3) 결과 저장 ────────────────────────────────
test_df["Clinician_management"] = outputs

  0%|          | 0/13 [00:00<?, ?it/s]

In [ ]:
test_df['Clinician'] = test_df['Clinician_summary'] + ' ' + test_df['Clinician_management']
test_submit = test_df[["Master_Index", "Clinician"]]
test_submit.to_csv(prefix + "test_submit.csv", index=False)
print("Saved → test_submit.csv")

Saved → test_submit.csv


In [ ]:
test_df['Clinician'][0]

'summary a 24 year old female presents with sharp pain in the right side of the nose that started 2 days ago and has been gradually worsening on assessment tenderness on palpation on the right side of the nasal bridge no visible signs of inflammation or infection vitals bp 129 81 mmhg pr 81 rr 20 t 36 8 spo2 94 management of the nasal bridge no visible signs of inflammation or infection vitals bp 129 81 mmhg pr 81 rr 20 t 36 8 spo2 94 question what could be the diagnosis of the patient bp 129 81 mmhg pr 81 rr 20 t 36 8 spo2 94 question what could be the diagnosis of the patient bp 129 81 mmhg pr 81 rr 20 t 36 8 spo2 94'

# one-model(summary + answer)

In [ ]:
hf_tgt = DatasetDict({
    "train": Dataset.from_pandas(train_df[['Prompt', 'Clinician']]),
    "validation": Dataset.from_pandas(val_df[['Prompt', 'Clinician']]),
})

from transformers import AutoTokenizer, T5ForConditionalGeneration
tokenizer = AutoTokenizer.from_pretrained("hossboll/clinical-t5")
model = AutoModelForSeq2SeqLM.from_pretrained("hossboll/clinical-t5")


#model = T5ForConditionalGeneration.from_pretrained("t5-small")
#tokenizer = T5Tokenizer.from_pretrained("t5-small")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
prefix = 'summarize'
def preprocess(examples):
    # prefix + Prompt
    inputs = [f"{prefix}: {txt}" for txt in examples["Prompt"]]
    targets = examples["Clinician"]

    # 입력 인코딩
    model_inputs = tokenizer(
        inputs,
        max_length=500,
        truncation=True,
        padding="max_length",
    )

    # 타깃 인코딩
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            targets,
            max_length=100,
            truncation=True,
            padding="max_length",
        )["input_ids"]

    # pad 토큰(0) → -100 로 치환
    labels = [
        [(tok if tok != tokenizer.pad_token_id else -100) for tok in seq]
        for seq in labels
    ]

    model_inputs["labels"] = labels
    return model_inputs


tokenized_data = hf_tgt.map(
    preprocess,
    batched=True,
    remove_columns=hf_tgt["train"].column_names,
)

Map:   0%|          | 0/1520 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/80 [00:00<?, ? examples/s]

In [ ]:
rouge = evaluate.load("rouge")

def compute_metrics(eval_pred):
    preds, labels = eval_pred
    if isinstance(preds, tuple): preds = preds[0]

    # logits → token IDs
    if preds.ndim == 3:
        preds = np.argmax(preds, axis=-1)

    # Tensor → numpy
    if isinstance(preds, torch.Tensor):  preds = preds.cpu().numpy()
    if isinstance(labels, torch.Tensor): labels = labels.cpu().numpy()

    # 음수 → pad, 범위 초과 → vocab_size-1 로 컷
    vocab_max = tokenizer.vocab_size - 1
    preds  = np.clip(np.where(preds  < 0, tokenizer.pad_token_id, preds),  0, vocab_max)
    labels = np.clip(np.where(labels < 0, tokenizer.pad_token_id, labels), 0, vocab_max)

    decoded_preds  = tokenizer.batch_decode(preds.tolist(),  skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels.tolist(), skip_special_tokens=True)

    rouge_scores = rouge.compute(predictions=decoded_preds,
                                 references=decoded_labels,
                                 use_stemmer=True)
    return {"rougeL_f1": float(rouge_scores["rougeL"])}

In [ ]:
#model = T5ForConditionalGeneration.from_pretrained("t5-small")

# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("hossboll/clinical-t5")
model = AutoModelForSeq2SeqLM.from_pretrained("hossboll/clinical-t5")


data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, label_pad_token_id=-100)

training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=8,
    num_train_epochs=10,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="steps",
    logging_steps = 5,
    predict_with_generate=True,
    generation_max_length=100,
    learning_rate=5e-5,
    fp16=True,
    report_to="none",
    generation_num_beams=4,
    label_smoothing_factor=0.1,
    weight_decay=0.01,
    warmup_ratio=0.05,
    optim = "adamw_torch",
    load_best_model_at_end=True,
    metric_for_best_model="rougeL_f1",
    greater_is_better=True,

)

trainer = Seq2SeqTrainer(
    model           = model,
    args            = training_args,
    train_dataset   = tokenized_data["train"],
    eval_dataset    = tokenized_data["validation"],
    tokenizer       = tokenizer,
    data_collator   = data_collator,
    compute_metrics = compute_metrics,
)

trainer.train()

<ipython-input-12-e2e110825e11>:38: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss,Rougel F1
1,5.510600,4.925733,0.343705
2,5.095600,4.639510,0.366839
3,4.958500,4.506069,0.370833
4,4.860400,4.416579,0.377266
5,4.285100,4.363068,0.387189
6,4.658200,4.322182,0.391816
7,4.657400,4.296944,0.391185
8,4.659500,4.277209,0.395044
9,4.615100,4.265596,0.393632


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


TrainOutput(global_step=470, training_loss=4.879814009970807, metrics={'train_runtime': 636.0937, 'train_samples_per_second': 23.896, 'train_steps_per_second': 0.739, 'total_flos': 1968800268288000.0, 'train_loss': 4.879814009970807, 'epoch': 9.8})

In [ ]:
save = dpath + "finetuned_clinical_t5"

trainer.save_model(save) # 모델 가중치 + config 저장
tokenizer.save_pretrained(save) # 토크나이저 vocab·특수토큰 저장

device = "cuda" if torch.cuda.is_available() else "cpu"
model_dir = dpath + "finetuned_clinical_t5"

tokenizer = AutoTokenizer.from_pretrained(model_dir, use_fast=True)
model = AutoModelForSeq2SeqLM.from_pretrained(model_dir).to(device)
model.eval()

In [ ]:
import torch, pandas as pd
from tqdm.auto import tqdm

# ── 0) 테스트 세트 로드 ──────────────────────────
test_df = pd.read_csv(dpath + 'test.csv')   # Prompt 열만 있다고 가정
test_df['Prompt'] = test_df['Prompt'].apply(split_prompt)

print("test size =", len(test_df))

# ── 1) 파라미터 & 디바이스 설정 ───────────────

MAX_INPUT = 512
MAX_GEN = 100
BEAMS = 4
LEN_PENALTY = 0.9
BATCH = 8                        # VRAM 여유에 맞게

# ── 2) 배치 추론 루프 ──────────────────────────
outputs = []
with torch.no_grad():
    for i in tqdm(range(0, len(test_df), BATCH)):
        batch_prompts = test_df["Prompt"].iloc[i:i+BATCH].tolist()
        batch_inputs  = [f"{prefix}: {p}" for p in batch_prompts]

        enc = tokenizer(
            batch_inputs,
            return_tensors="pt",
            padding=True,           # 가변 길이 배치
            truncation=True,
            max_length=MAX_INPUT
        ).to(device)

        gen_ids = model.generate(
            **enc,
            max_length=MAX_GEN,
            num_beams=BEAMS,
            length_penalty=LEN_PENALTY,
        )

        batch_out = tokenizer.batch_decode(
            gen_ids, skip_special_tokens=True
        )
        outputs.extend(batch_out)

# ── 3) 결과 저장 ────────────────────────────────
test_df["Clinician"] = outputs
#test_df['Clinician'] = test_df['Clinician'].str.replace('[clinical_summary]', '')
#test_df['Clinician'] = test_df['Clinician'].str.replace('[clinical_answer]', '')
test_submit = test_df[["Master_Index", "Clinician"]]
test_submit.to_csv(prefix + "test_submit.csv", index=False)
print("Saved → test_submit.csv")

test size = 100


  0%|          | 0/13 [00:00<?, ?it/s]

Saved → test_submit.csv


In [ ]:
test_df['Prompt'][3]

'context: i am a nurse working in a dispensaries and private clinics in kakamega county in kenya a 6 years old girl is brought to opd with history of twitching the father reports the baby was asleep when she suddenly woke up from sleep and started twitching no history of ailments reports the baby was very okay during the day on examination the baby is stable no twitching noted vital signs normal and mrdt done negative question: what could be the problem with the girl what is the diagnosis and which medication do i prescribeanswer:'

In [ ]:
test_df['Clinician'][6]

'summary a 14 year old female complains of right lower limb pain for one day she is reported to have had a fell while playing football vitals are normal questions what treatment can be diagnosed what other investigations could be diagnosed'